In [1]:
'''Code to train Autoencoder for fingerprints'''
import os
import numpy as np
import pandas as pd
import tensorflow as tf

#gpus = tf.config.experimental.list_physical_devices('GPU')

#tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])

from tensorflow import keras
from tensorflow.keras.applications import vgg16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2D,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D,LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint

import matplotlib.pyplot as plt
from skimage.transform import resize

In [2]:
img_ncols = 224
img_nrows = 224

def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

def total_variation_loss(x):
    a = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :]
    )
    b = tf.square(
        x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))


# Build a VGG19 model loaded with pre-trained ImageNet weights
model = vgg16.VGG16(weights="imagenet", include_top=False)

layers = [model.layers[1], model.layers[4], model.layers[7], model.layers[11], model.layers[15]] 

# Get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in layers])

# Set up a model that returns the activation values for every layer in
# VGG16 (as a dict).
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

total_variation_weight = 1e-6
style_weight = 1e-6

In [3]:
layer_names =[]
for layer in layers:
    layer_names.append(layer.name)

In [23]:
'''Computing Perceptual loss(Style loss) for several layers'''
def compute_loss(Y_true, Y_pred):
    # returning back to normal 0-1
    Y_true = reverseimage(Y_true)
    Y_pred = reverseimage(Y_pred)
    #b_s = Y_true.shape[0]
    b_s = 32
    #resizing to the 224x224
    #Y_true = resize(Y_true, (b_s,224, 224))
    #Y_pred = resize(Y_pred, (b_s,224, 224))
    
    # preprocess for vgg16:
    Y_true = preprocess_input(Y_true)
    Y_pred = preprocess_input(Y_pred)
    
    input_tensor = tf.concat(
        [Y_true, Y_pred], axis=0
    )
    features = feature_extractor(input_tensor)
    # Initialize the loss
    loss = tf.zeros(shape=())

    for layer_name in layer_names:
        layer_features = features[layer_name]
        for i in range(b_s):
            style_true_features = layer_features[i, :, :, :]
            style_pred_features = layer_features[i+b_s, :, :, :]
            sl = style_loss(style_true_features, style_pred_features)
            loss += (style_weight / len(layer_names)) * sl

    # Add total variation loss
    #loss += total_variation_weight * total_variation_loss(Y_pred)
    return loss

In [5]:
data_pd = pd.read_pickle('/home/cc/Data/Zips/cm_index_df_fixed.p')
data_pd['FilePath']= data_pd.FilePath.apply(lambda x: os.path.join('/home/cc/Data/Images',x))
data_live = data_pd[data_pd.live_spoof == 0] # only live samples
data_live['Collection'] = data_live.CaptureID.apply(lambda x: x[0:3])
resize_df = data_live[data_live.Collection =='Pre']
resize_df.shape

/home/cc/anaconda3/envs/Clarkson/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(236345, 21)

In [6]:
def rescaleimage(image):
    image=(image-127.5)/127.5
    return image
def reverseimage(image):
    image=(image/2)+0.5
    return image

In [7]:
img_rows = 512
img_cols = 512
channels = 3 # I am using channel last models
img_shape = (img_rows, img_cols, channels)
latent_dim = 512
batch_size=32

inputTrain ='/home/cc/Data/Images'

datagen=ImageDataGenerator(preprocessing_function=rescaleimage)
#datagen=ImageDataGenerator(preprocessing_function=rescaleimage, horizontal_flip=True, vertical_flip=True)
image_generator=datagen.flow_from_dataframe(
    directory = inputTrain,
    dataframe = resize_df, 
    color_mode='rgb',
    x_col = 'FilePath',
    batch_size= batch_size,
    target_size=(img_rows,img_cols),
    class_mode='input',
    shuffle=True
    )

Found 236345 validated image filenames.


In [8]:
initialize=RandomNormal(mean=0.0, stddev=0.02, seed=None)
'''
G_ENC
'''
CAE = Sequential()
CAE.add(Conv2D(16, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, input_shape=(512,512,3), padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(32, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(64, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(128, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(256, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(512, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Conv2D(1024, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

CAE.add(Flatten())
CAE.add(Dense(512))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(LeakyReLU(alpha=0.2))

'''
G-DEC
'''
CAE.add(Dense(1024 * 4 * 4))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Reshape((4, 4, 1024)))
CAE.add(Conv2DTranspose(512, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))        
CAE.add(Conv2DTranspose(256, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(128, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))        
CAE.add(Conv2DTranspose(64, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(32, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(16, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('relu'))
CAE.add(Conv2DTranspose(3, kernel_size=4, strides=2, kernel_regularizer=l2(0.0005), kernel_initializer=initialize, padding="same"))
CAE.add(BatchNormalization(momentum=0.8))
CAE.add(Activation('tanh'))

optimizer=Adam(lr=0.0001, beta_1=0.5, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=True)

In [38]:
CAE.compile(loss=compute_loss, optimizer=optimizer)

()


In [36]:
'''Computing Perceptual loss(Style loss) for several layers'''
def compute_loss(Y_true, Y_pred):
    # returning back to normal 0-1
    loss = tf.reduce_mean(Y_true) - tf.reduce_mean(Y_pred)
    print(loss.shape)
    #loss= tf.constant(bs, dtype=tf.float32)
    return loss

In [39]:
if not os.path.exists('/home/cc/Data/Tests/logs'):
    os.makedirs('/home/cc/Data/Tests/logs')
if not os.path.exists('/home/cc/Data/Tests/MC'):
    os.makedirs('/home/cc/Data/Tests/MC')
    
TS = TensorBoard(log_dir='/home/cc/Data/Tests/logs', histogram_freq=0)
MC = ModelCheckpoint(os.path.join('/home/cc/Data/Tests','MC'), 
                        monitor='loss', verbose=1, save_best_only=True, 
                        save_weights_only=True, mode='auto', save_freq="epoch")

CAE.fit(image_generator, steps_per_epoch=len(image_generator), epochs=5, verbose=1, callbacks=[TS, MC])

  ...
    to  
  ['...']
Train for 7386 steps
Epoch 1/5
()
  12/7386 [..............................] - ETA: 55:00 - loss: 3.6796
Epoch 00001: loss improved from inf to 3.66794, saving model to /home/cc/Data/Tests/MC
  12/7386 [..............................] - ETA: 1:24:05 - loss: 3.6796

KeyboardInterrupt: 

In [ ]:
latest = tf.train.latest_checkpoint('/home/cc/Data/CAE')
CAE.load_weights(latest)

In [ ]:
x1 = image_generator.next()[0]
#x = CAE.predict(x1)

In [ ]:
x1.shape

In [ ]:
plt.imshow(x1[1,:,:,:])

In [ ]:
input_tensor = tf.concat(
        [x1, x1], axis=0
    )

In [ ]:
compute_loss(x1,x1)

In [ ]:
img1 = []
img2 = []
N = [0,1, 2, 3, 4, 5]
for n in N:
    img1.append(x1[n,:,:,:])
    img2.append(x[n,:,:,:])

In [ ]:
for n in N:
    img1[n] = img1[n]/2
    img1[n] = img1[n]+0.5
    img2[n] = img2[n]/2
    img2[n] = img2[n]+0.5

In [ ]:
import matplotlib.pyplot as plt
i=1
plt.figure(figsize=(10, 10))
for n in N:
    plt.subplot(6,2,i)
    plt.imshow(img1[n])
    i= i+1
    plt.subplot(6,2,i)
    plt.imshow(img2[n])
    i=i+1

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img1[1], cmap='gray')

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img2[1], cmap='gray')